In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
df = pd.read_csv("../../completed/df_stats.csv", index_col=0)

## 2.1 Analisi Statistica e Regressione (Inference)



In questa parte, l'attenzione è posta sulla significatività statistica e sull'interpretazione dei coefficienti. Non stiamo ancora cercando di fare la "miglior predizione possibile", ma di capire come le variabili si influenzano a vicenda.

**Selezione delle variabili**  
Come sappiamo il **target** è Price, e il resto sono i nostri **predittori**. Naturalmente cercheremo di investigare sulla significatività statistica di ogni coefficiente, e quindi potremmo dover cambiare alcune scelte.

Applichiamo utilizzando la libreria **statsModels** il nostro modello di **Regressione Lineare.**

In [3]:
y_inference = df['Price']

vars_da_escludere = ['Price'] 
X_inference = df.drop(columns=vars_da_escludere)

X_inference = sm.add_constant(X_inference)

modello = sm.OLS(y_inference, X_inference).fit()

print(modello.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     2171.
Date:                Fri, 26 Dec 2025   Prob (F-statistic):               0.00
Time:                        09:29:04   Log-Likelihood:                 1362.4
No. Observations:                5710   AIC:                            -2651.
Df Residuals:                    5673   BIC:                            -2405.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.8504    

**1. Analisi della Bontà di Adattamento (R-squared)**  
R-squared (0.932): Il modello spiega il 93.2% della varianza totale dei prezzi. Questo indica un adattamento eccellente ai dati.  
Prob (F-statistic) (0.00): Indica che il modello nel suo complesso è statisticamente significativo (non è un risultato casuale).  
Adj. R-squared (0.932): È identico all'R-squared semplice. Significa che non abbiamo "gonfiato" il modello con variabili inutili (spazzatura). Tutte le variabili aggiunte portano informazione.

**2. Diagnostica: Il problema della Multicollinearità**  
Nelle note:  
The condition number is large, 1.64e+03. This might indicate that there are strong multicollinearity...

Avevamo ragione a temere Engine + Power. Il numero è alto, il che significa che le due variabili "litigano" un po'. Tuttavia, guardando i P-value, entrambe sono rimaste significative (0.000) e hanno coefficienti positivi.  
**Interpretazione**: Nonostante la collinearità, il modello beneficia dell'inclusione di entrambe le variabili per massimizzare la precisione (R2), sebbene l'interpretazione dei singoli coefficienti di potenza e cilindrata vada fatta con cautela.  

**3. Interpretazione dei Coefficienti**  
- A. Variabili di Usura (Age, Km)  
Age (Coef: -0.0962):  
Mantenendo costanti le altre variabili, ogni anno di invecchiamento dell'auto è associato a una riduzione del prezzo di circa il 9.6%. Questo rispecchia fedelmente il deprezzamento annuale di mercato.  
Kilometers_Driven (Coef: -0.0717):  
L'usura chilometrica ha un impatto negativo significativo: all'aumentare dei chilometri, il valore diminuisce.  

- B. Variabili di Performance (Power, Engine)  
Power (Coef: 0.6838):  
La potenza è il predittore positivo più forte. Un motore più potente è associato drasticamente a un prezzo più alto.  

- C. Variabili Categoriche (Brand e Trasmissione)
Le variabili Dummy funzionano per confronto con una base. Quale Brand manca nella lista? Audi. Quindi tutti i numeri sono rispetto ad un'Audi:  
Brand_BMW (P-value 0.201, Coef -0.02):  
Il coefficiente non è statisticamente significativo (P > 0.05). Questo significa che, a parità di età e potenza, non c'è differenza di prezzo statistica tra una BMW e un'Audi (il brand di riferimento). Il mercato le valuta uguali.  
Brand_Maruti (Coef: -0.4893):  
Il coefficiente è negativo e significativo. Rispetto al brand di riferimento (Audi), una Maruti vale circa il 49% in meno a parità di altre caratteristiche.  
Transmission (Coef: 0.0745):
Passare dal cambio Manuale (0) all'Automatico (1) comporta un incremento di prezzo del 7.5% circa, confermando il valore aggiunto della trasmissione automatica.  

**4. Analisi dei P-Value**  
Promossi (Significativi, P < 0.05): Age, Km, Fuel, Transmission, Power, Engine, Seats e la maggior parte dei Brand economici (Maruti, Hyundai, ecc.). Queste variabili devono restare nel modello.  

Bocciati o Neutri (Non Significativi, P > 0.05):  
Brand_BMW (0.201), Brand_Mercedes-Benz (0.328), Location_Jaipur, Chennai, Kochi.  
Significato: Questo non vuol dire che "non contano", ma che non sono diverse dalla media o dal riferimento (Audi). E inoltre ha senso che BMW e Mercedes siano simili ad Audi.

**5. Analisi degli Intervalli di Confidenza ([0.025 0.975])**  
Diamo una visione generale su quelli più rilevanti:  
Guardiamo la riga di Age:  
Coefficiente stimato: -0.0962. Intervallo: [-0.099, -0.094].  
Commento: L'intervallo è strettissimo (i due numeri sono vicinissimi). Questo indica una grandissima precisione della stima. Siamo estremamente certi che il deprezzamento annuo sia tra il 9.4% e il 9.9%.  
Guardiamo la riga di Brand_BMW:  
Intervallo: [-0.057, 0.012].  
Commento: L'intervallo contiene lo zero (va da negativo a positivo). Ecco perché il P-value è alto, non siamo sicuri se il brand BMW aggiunga o tolga valore rispetto alla base. Statisticamente, l'effetto è nullo.  
Power: L'intervallo è [0.645, 0.723].  
Interpretazione: Anche per la potenza la stima è solida. L'effetto positivo sul prezzo è netto e ben definito.  

La maggior parte degli intervalli di confidenza sono ristretti e non includono lo zero, il che indica che le stime dei coefficienti sono precise e affidabili. Le uniche eccezioni, riguardano brand di lusso comparabili al riferimento o location specifiche, dove l'effetto prezzo è statisticamente trascurabile.

## 2.2 Analisi Predittiva e Machine Learning

### 2.2.1 Definizione del Problema e del Valore Pratico

**Titolo del Progetto**: Algoritmo di Pricing per il mercato delle automobili ("AutoPrice AI").

**1. Scenario Applicativo**  
Immaginiamo di integrare questo modello predittivo all'interno di una piattaforma web di compravendita auto o nel gestionale interno di una grande concessionaria di auto usate. Quando un utente inserisce i dati tecnici di una vettura, il sistema utilizza l'algoritmo di **Machine Learning** per calcolare in tempo reale il **Price** dell'auto, senza intervento umano.

**2. Utilità del Problema (Perché risolverlo?)**  
Il mercato delle auto usate soffre di una forte asimmetria informativa:  
- Chi vende spesso non sa quanto vale la sua auto: rischia di svenderla o di proporla a un prezzo fuori mercato.  
- Chi compra ha paura di pagare troppo o di essere truffato.  
- Può essere utile per i concessionari, per stabilire un prezzo che è più basato sul mercato corrente.  
Questo strumento risolve il problema fornendo una valutazione oggettiva, immediata e basata sui dati reali di migliaia di transazioni simili, aggiungendo certezze al soggetto interessato. 

**3. Quantificazione del Valore Aggiunto**  
L'implementazione di questo modello porterebbe vantaggi tangibili:  
- Risparmio tempo: Riduzione del tempo di valutazione da 20-30 minuti (ricerca manuale) a < 1 secondo. Per una concessionaria che valuta molteplici auto al mese, questo equivale a risparmiare moltissimo.  
- Incremento del profitto: Evitando errori di sovrastima nell'acquisto di auto usate, il concessionario protegge il proprio margine di profitto.    
- Fiducia del cliente: Una piattaforma online che mostra un "Prezzo Consigliato dall'AI" aumenta la trasparenza, incrementando la probabilità di contatto tra venditore e acquirente.

Data la natura del nostro contesto (pricing), si è scelto di mantenere l'approccio di Regressione per predire il valore dell'auto, in quanto abbiamo già ricevuto dei risultati eccellenti. 

### 2.2.2 Setup Sperimentale

**1. Preparazione e Data Splitting**  
Dividiamo i dati in Training Set (80%) e Test Set (20%).  
*Nota*: La validazione la faremo dopo usando la Cross-Validation automatica dentro i modelli, quindi per ora ci basta dividere in due (Train/Test).

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
X = df.drop(columns=['Price']) 

y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensioni Training Set: {X_train.shape}")
print(f"Dimensioni Test Set:     {X_test.shape}")

Dimensioni Training Set: (4568, 36)
Dimensioni Test Set:     (1142, 36)


**2. Preprocessing per ML (Scaling)**  
Qui applichiamo lo StandardScaler.

In [7]:
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_final = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_final = pd.DataFrame(X_test_scaled, columns=X.columns)

X_train_final.head()

,Age,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Brand_BMW,...,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune
0,0.207501,0.622293,-0.923658,1.550443,-0.433436,-0.221204,0.753583,1.263859,-0.357044,-0.214539,...,-0.248163,-0.291609,2.862018,-0.32689,-0.369596,-0.269355,-0.352944,-0.321116,-0.39072,-0.324423
1,-0.808795,-1.089762,1.082652,-0.644977,-0.433436,0.751562,-0.743476,-0.551891,-0.357044,-0.214539,...,-0.248163,-0.291609,-0.349404,3.05913,-0.369596,-0.269355,-0.352944,-0.321116,-0.39072,-0.324423
2,-0.131264,-0.184139,1.082652,1.550443,-0.433436,-1.923545,0.478176,1.017559,-0.357044,-0.214539,...,-0.248163,-0.291609,-0.349404,3.05913,-0.369596,-0.269355,-0.352944,-0.321116,-0.39072,-0.324423
3,0.546266,-0.139732,-0.923658,-0.644977,-0.433436,0.508370,-0.274000,-1.039218,-0.357044,-0.214539,...,-0.248163,-0.291609,-0.349404,-0.32689,-0.369596,-0.269355,-0.352944,3.114144,-0.39072,-0.324423
4,-0.131264,0.309645,1.082652,1.550443,-0.433436,-0.464396,-0.070147,0.306023,-0.357044,-0.214539,...,-0.248163,-0.291609,-0.349404,-0.32689,-0.369596,-0.269355,2.833314,-0.321116,-0.39072,-0.324423


### 2.2.3 Modellazione e Selezione

In [10]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [13]:
# --- MODELLO 1: Linear Regression emplice
lin_reg = LinearRegression()
# Qui non c'è grid search perché non ha iperparametri da regolare
lin_reg.fit(X_train_final, y_train)
score_lin = lin_reg.score(X_test_final, y_test)
print(f"1. Linear Regression R2: {score_lin:.4f}")


# --- MODELLO 2: Ridge Regression (Per gestire la collinearità Engine/Power) ---
# Usiamo GridSearch per trovare l'alpha giusto
ridge_params = {'alpha': [0.01, 0.1, 1, 10, 50, 100]}
ridge_grid = GridSearchCV(Ridge(), ridge_params, cv=5, scoring='r2')
ridge_grid.fit(X_train_final, y_train)

print(f"\n2. Ridge Regression Best R2 (CV): {ridge_grid.best_score_:.4f}")
print(f"   Miglior alpha: {ridge_grid.best_params_}\n")


# --- MODELLO 3: Polynomial Regression
# ATTENZIONE: Usiamo degree=2. Se metti 3 esplode il PC (troppe colonne).
# Creiamo una Pipeline: Prima crea i quadrati delle variabili poi fa la Regressione Ridge
# Usiamo Ridge dentro la polinomiale per evitare che i coefficienti esplodano

poly_pipeline = make_pipeline(PolynomialFeatures(degree=2), Ridge())

# Cerchiamo l'alpha anche qui
poly_params = {'ridge__alpha': [1, 10, 100]}
poly_grid = GridSearchCV(poly_pipeline, poly_params, cv=5, scoring='r2')

poly_grid.fit(X_train_final, y_train)

print(f"3. Polynomial Regression Best R2 (CV): {poly_grid.best_score_:.4f}")
print(f"   Miglior alpha per la Polynomial: {poly_grid.best_params_}")

1. Linear Regression R2: 0.9429

2. Ridge Regression Best R2 (CV): 0.9277
   Miglior alpha: {'alpha': 1}

3. Polynomial Regression Best R2 (CV): 0.9381
   Miglior alpha per la Polynomial: {'ridge__alpha': 100}


**Scelta degli Algoritmi**:  

Regressione Lineare: Utilizzata come base di riferimento per valutare le performance minime accettabili.  

Ridge Regression: Selezionata specificamente per gestire la multicollinearità rilevata nell'analisi esplorativa (tra Engine e Power). La regolarizzazione L2 penalizza i coefficienti troppo alti, riducendo la varianza del modello senza eliminare variabili.  

Regressione Polinomiale (Grado 2): Introdotta per catturare possibili le relazioni non lineari, nonostante le trasformazioni logaritmiche, osservate nei grafici (es. la crescita esponenziale del prezzo rispetto alla potenza).

### 2.2.4 Valutazione e Confronto

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [15]:
models_to_evaluate = {
    "Linear Regression": lin_reg,
    "Ridge Regression": ridge_grid.best_estimator_,
    "Polynomial Regression (Deg 2)": poly_grid.best_estimator_
}

results_list = []

print("--- VALUTAZIONE SUL TEST SET")

for name, model in models_to_evaluate.items():
    y_pred_test = model.predict(X_test_final)
    
    # Calcola le metriche
    r2 = r2_score(y_test, y_pred_test)
    mse = mean_squared_error(y_test, y_pred_test)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred_test)
    
    # Salva i risultati
    results_list.append({
        "Model": name,
        "R2 Score": r2,
        "RMSE (Log Scale)": rmse,
        "MAE (Log Scale)": mae
    })

# Crea la classifica finale
df_results = pd.DataFrame(results_list).sort_values(by='R2 Score', ascending=False)
print(df_results)

--- VALUTAZIONE SUL TEST SET
                           Model  R2 Score  RMSE (Log Scale)  MAE (Log Scale)
2  Polynomial Regression (Deg 2)  0.956100          0.158417         0.118733
1               Ridge Regression  0.942901          0.180668         0.139884
0              Linear Regression  0.942900          0.180671         0.139885


**Confronto Critico**  
- Best Model: Regressione Polinomiale (Grado 2). È il modello indiscusso, raggiungendo un $R^2$ Score di 0.956 e un RMSE di 0.158 sul Test Set. Questo miglioramento netto rispetto ai modelli lineari dimostra che la relazione tra le caratteristiche e il Price non è puramente lineare, ma presenta curvature che solo il modello polinomiale ha saputo catturare.  
- Regressione Lineare e la Ridge Regression hanno ottenuto risultati quasi identici ($R^2 \approx 0.943$). La GridSearch per la Ridge ha selezionato un parametro di penalizzazione molto basso ($\alpha=1$), indicando che il modello non necessitava di una forte regolarizzazione per performare bene. Tuttavia, la Ridge rimane preferibile per la sua capacità teorica di gestire meglio la multicollinearità.  
- È interessante notare che le performance sul Test Set ($0.94-0.95$) sono risultate leggermente superiori a quelle stimate in Cross-Validation ($0.92-0.94$). Questo fenomeno esclude problemi di Overfitting e conferma che il modello è in grado di generalizzare eccellentemente su nuovi dati mai visti.

## 2.3 Conclusioni Parziali

**1. Le principali relazioni statistiche scoperte**  
L'analisi inferenziale ha confermato la validità delle teorie economiche sul deprezzamento e ha quantificato l'impatto delle features.
- È stata trovata una relazione negativa forte e precisa. A parità di altre condizioni, ogni anno di età riduce il valore dell'auto di circa il 9.6%.
- La Potenza (Power) è risultata essere il predittore positivo più influente. L'analisi ha evidenziato che il mercato premia la performance in modo non-lineare.
- La trasmissione Automatica offre un incremento di valore netto stimato intorno al 7.5% rispetto alla manuale.
- Brands: I coefficienti delle variabili dummy hanno delineato una chiara gerarchia di mercato. I brand più economici mostrano valutazioni inferiori del 45-50% rispetto ai brand più costosi (Audi), mentre non vi è evidenza statistica significativa di differenze di prezzo tra i giganti del lusso (Audi, BMW, Mercedes), che il mercato percepisce come equivalenti.
**2. Performance del miglior modello e applicabilità**  
Per la fase predittiva sono stati confrontati modelli lineari (OLS, Ridge) e non lineari.
- La Regressione Polinomiale di Grado 2 (con regolarizzazione Ridge) ha superato tutti gli altri algoritmi, raggiungendo un $R^2$ Score di 0.956 sul Test Set. Il modello è in grado di spiegare il 95.6% della varianza dei prezzi. L'errore medio (RMSE) di 0.158 indica un margine di errore percentuale medio intorno al 15-16%, un risultato eccellente.
- Applicabilità nello scenario ipotizzato: Il modello sviluppato è pronto per essere integrato in una piattaforma per concessionari o portali web. Il modello permette in tempo reale risultati oggettivi basati sui dati storici, riducendo drasticamente i tempi d'attesa.